In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv("C:/Users/80071/Downloads/spam.csv",encoding='latin-1')

data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":"class", "v2":"text"})
data.head()

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
data.describe().transpose()
data['class'].value_counts()

ham     4825
spam     747
Name: class, dtype: int64

In [7]:
data['length'] = data['text'].apply(len)
data.head()

,class,text,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [8]:
# Function to remove stop words , then converting all remaining character to lower case and then using snowball stemmer converting 
#all the words to their roo form . for eg: tasting and tasted to taste 
def pre_process(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    words = ""
    for i in text:
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "
    return words

In [10]:
textFeatures = data['text'].copy()
textFeatures = textFeatures.apply(pre_process)
print(textFeatures)

0       go jurong point crazi avail bugi n great world...
1                                  ok lar joke wif u oni 
2       free entri 2 wkli comp win fa cup final tkts 2...
3                    u dun say earli hor u c alreadi say 
4              nah dont think goe usf live around though 
                              ...                        
5567    2nd time tri 2 contact u u å£750 pound prize 2...
5568                             ì b go esplanad fr home 
5569                             piti mood soani suggest 
5570    guy bitch act like id interest buy someth els ...
5571                                      rofl true name 
Name: text, Length: 5572, dtype: object


In [11]:
vectorizer = TfidfVectorizer("english")
features = vectorizer.fit_transform(textFeatures)
print(features)

  (0, 7574)	0.19449555011263384
  (0, 1138)	0.3501112803835596
  (0, 3342)	0.164475816719243
  (0, 2006)	0.29577542446330046
  (0, 1724)	0.33421906130459
  (0, 4204)	0.29577542446330046
  (0, 7776)	0.2379272168522122
  (0, 3380)	0.19449555011263384
  (0, 1726)	0.29577542446330046
  (0, 1327)	0.2662313123328933
  (0, 2223)	0.2711370860940892
  (0, 5539)	0.23906338437276378
  (0, 4063)	0.3501112803835596
  (0, 3289)	0.14081218308843002
  (1, 5200)	0.5633086751818669
  (1, 7688)	0.44480400570972006
  (1, 4029)	0.47731294876998304
  (1, 4239)	0.42078899608869724
  (1, 5170)	0.2827396376113674
  (2, 71)	0.23416578360011273
  (2, 1210)	0.16749632565308856
  (2, 5814)	0.23416578360011273
  (2, 7274)	0.12583555023089024
  (2, 5770)	0.23416578360011273
  (2, 5855)	0.16104729275511298
  :	:
  (5568, 2797)	0.6620015064495088
  (5568, 3097)	0.5769730291899227
  (5568, 3634)	0.3752780670596876
  (5568, 3289)	0.29668586321213597
  (5569, 6454)	0.5375461940984255
  (5569, 5475)	0.5375461940984255
  (

In [14]:
features.shape

(5572, 8037)

In [15]:


features_train, features_test, labels_train, labels_test = train_test_split(features, data['class'], test_size=0.3, random_state=111)

In [21]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

svc = SVC(kernel='linear', gamma=3)
svc.fit(features_train, labels_train)
prediction = svc.predict(features_test)
accuracy_score(labels_test,prediction)

0.9814593301435407

In [25]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB(alpha=0.3)
mnb.fit(features_train, labels_train)
prediction = mnb.predict(features_test)
accuracy_score(labels_test,prediction)

0.9826555023923444

In [26]:
# Using Neural Network
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(12, input_dim=8037, activation='relu'))
model.add(Dense(8037, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
new_labels_tarin=[]
for i in labels_train:
    if i == "ham":
        new_labels_tarin.extend('1')
    else:
        new_labels_tarin.extend('0')
model.fit(features_train, pd.Series(new_labels_tarin), epochs=150, batch_size=10)
predictions = model.predict(features_test)
rounded = [round(x[0]) for x in predictions]
from sklearn.metrics import accuracy_score

new_labels_test=[]
for i in labels_test:
    if i == "ham":
        new_labels_test.append(1.0)
    else:
        new_labels_test.append(0.0)
accuracy_score(new_labels_test,rounded)

Using TensorFlow backend.







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/150
3900/3900 [==============================] - 2s 543us/step - loss: 0.2087 - acc: 0.9333
Epoch 2/150
3900/3900 [==============================] - 1s 258us/step - loss: 0.0372 - acc: 0.9923
Epoch 3/150
3900/3900 [==============================] - 1s 278us/step - loss: 0.0090 - acc: 0.9990
Epoch 4/150
3900/3900 [==============================] - 1s 270us/step - loss: 0.0034 - acc: 0.9995
Epoch 5/150
3900/3900 [==============================] - 1s 266us/step - loss: 0.0024 - acc: 0.9997
Epoch 6/150
3900/3900 [==============================] - 1s 254us/step - loss: 0.0021 - acc: 0.9997
Epoch 7/150
3900/3900 [==============================] - 1s 252us/step - loss: 0.0020 - acc: 0.9997
Epoch 8/150
3900/3900 [==============================] - 1s 253us/step - loss: 0.0020 - acc: 0.9997
Epoch 9/150
3900/3900 [==============================] - 1s 253us/step - loss: 0.0019 - acc: 0.9997


3900/3900 [==============================] - 1s 290us/step - loss: 4.9787e-07 - acc: 1.0000
Epoch 66/150
3900/3900 [==============================] - 1s 286us/step - loss: 4.2072e-07 - acc: 1.0000
Epoch 67/150
3900/3900 [==============================] - 1s 273us/step - loss: 3.6388e-07 - acc: 1.0000
Epoch 68/150
3900/3900 [==============================] - 1s 331us/step - loss: 3.1729e-07 - acc: 1.0000
Epoch 69/150
3900/3900 [==============================] - 1s 292us/step - loss: 2.7894e-07 - acc: 1.0000
Epoch 70/150
3900/3900 [==============================] - 1s 292us/step - loss: 2.5146e-07 - acc: 1.0000
Epoch 71/150
3900/3900 [==============================] - 1s 335us/step - loss: 2.2824e-07 - acc: 1.0000
Epoch 72/150
3900/3900 [==============================] - 1s 323us/step - loss: 2.0848e-07 - acc: 1.0000
Epoch 73/150
3900/3900 [==============================] - 1s 331us/step - loss: 1.9305e-07 - acc: 1.0000
Epoch 74/150
3900/3900 [==============================] - 1s 294us/s

3900/3900 [==============================] - 1s 331us/step - loss: 1.1667e-07 - acc: 1.0000
Epoch 143/150
3900/3900 [==============================] - 1s 335us/step - loss: 1.1667e-07 - acc: 1.0000
Epoch 144/150
3900/3900 [==============================] - 1s 315us/step - loss: 1.1667e-07 - acc: 1.0000
Epoch 145/150
3900/3900 [==============================] - 1s 292us/step - loss: 1.1667e-07 - acc: 1.0000
Epoch 146/150
3900/3900 [==============================] - 1s 304us/step - loss: 1.1667e-07 - acc: 1.0000
Epoch 147/150
3900/3900 [==============================] - 1s 301us/step - loss: 1.1667e-07 - acc: 1.0000
Epoch 148/150
3900/3900 [==============================] - 1s 314us/step - loss: 1.1667e-07 - acc: 1.0000
Epoch 149/150
3900/3900 [==============================] - 1s 307us/step - loss: 1.1667e-07 - acc: 1.0000
Epoch 150/150
3900/3900 [==============================] - 1s 302us/step - loss: 1.1667e-07 - acc: 1.0000


0.9832535885167464